In [1]:
import torch
import numpy as np
import pandas as pd
import random
from tqdm import tqdm
from datasets import load_dataset
import re
import wandb
from datetime import datetime
from utils import get_device, load_model, load_gsm8k, extract_answer, create_cot_prompt, generate_answer_hf, is_correct_check

# Configuration
config = {
    # Model settings
    "model_name": "gpt2",  # Options: "gpt2", "gpt2-medium", "gpt2-large", "gpt2-xl"
    "max_length": 512,  # Increased for few-shot CoT
    
    # Dataset settings
    "num_samples": 100,  # Set to None to use the full dataset
    "n_shot": 4,  # Number of examples for few-shot prompting
    
    # Generation settings
    "temperature": 0.7,
    "top_p": 0.9,
    "num_beams": 4,
    
    # Experiment tracking
    "run_name": f"gpt2-gsm8k-8shot-cot-{datetime.now().strftime('%Y%m%d-%H%M%S')}",
    "tags": ["gsm8k", "evaluation", "gpt2", "few-shot", "cot"]
}


# Initialize Weights & Biases with config
wandb.init(
    project="gpt2-gsm8k-benchmark", 
    name=config["run_name"],
    config=config,
    tags=config["tags"]
)


# Set device
DEVICE = get_device()

print(f"Using device: {DEVICE}")

# Main evaluation function
def evaluate_gsm8k():
    # Load dataset
    train_set, test_set = load_gsm8k(config)
    
    # Prepare training examples list for few-shot prompting
    train_examples = []
    for ex in train_set:
        train_examples.append({
            "question": ex["question"],
            "answer": ex["answer"]
        })
    
    # Load model and tokenizer
    model, tokenizer = load_model(config, DEVICE)
    
    results = []
    correct_count = 0
    
    for idx, example in enumerate(tqdm(test_set, desc="Evaluating")):
        question = example["question"]
        
        # Extract target answer, forcing it to be a float
        target_answer = extract_answer(example["answer"])
        print(f"\n\n{'-'*80}")
        print(f"Example {idx+1}:")
        print(f"Question: {question}")
        print(f"Target answer: {target_answer}")
        
        # Create and print the full CoT prompt
        prompt = create_cot_prompt(train_examples, question, n_shot=config["n_shot"])
        print(f"\nFull CoT Prompt:\n{'-'*40}\n{prompt}\n{'-'*40}\n")

        # Generate and print the full model response
        model_response = generate_answer_hf(model, tokenizer, prompt, config, DEVICE, model_type="gpt2")
        print(f"\nFull Model Response:\n{'-'*40}\n{model_response}\n{'-'*40}\n")
        
        # Extract answer from response
        predicted_answer = extract_answer(model_response)
        print(f"Extracted predicted answer: {predicted_answer}")
        
        # Fix for the integer error: Ensure both are converted to float for comparison
        is_correct = is_correct_check(predicted_answer, target_answer)
        
        print(f"Is correct? {is_correct}")
        
        if is_correct:
            correct_count += 1
        
        results.append({
            "question": question,
            "target_answer": target_answer,
            "model_response": model_response,
            "predicted_answer": predicted_answer,
            "is_correct": is_correct
        })
        
        # Log to wandb
        wandb.log({
            "running_accuracy": correct_count / (idx + 1),
            "example_idx": idx
        })
        
        # Early stop option for debugging (remove for full evaluation)
        if idx == 3 and 'debug' in config and config['debug']:
            break
    
    # Calculate and log final accuracy
    accuracy = correct_count / len(results)
    print(f"\nFinal accuracy: {accuracy:.2%} ({correct_count}/{len(results)})")
    
    # Log final metrics to wandb
    wandb.log({
        "final_accuracy": accuracy,
    })
    
    # Save detailed results to CSV
    results_df = pd.DataFrame(results)
    results_file = f"{config['model_name']}_gsm8k_{config['n_shot']}shot_results.csv"
    results_df.to_csv(results_file, index=False)
    print(f"Detailed results saved to {results_file}")
    
    # Log results file to wandb
    wandb.save(results_file)
    
    return accuracy, results

if __name__ == "__main__":
    try:
        accuracy, results = evaluate_gsm8k()
        print(f"Evaluation completed successfully with accuracy: {accuracy:.2%}")
    except Exception as e:
        print(f"Error during evaluation: {e}")
        wandb.log({"error": str(e)})
    finally:
        wandb.finish()

/Users/jonathan/Library/Mobile Documents/com~apple~CloudDocs/Master/Master Thesis/math-reasoning-in-language-models/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: jonathantiedchen (master_thesis_math_lm) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Using device: mps
Loading GSM8K dataset...
Loaded 7473 training examples and 100 test examples
Loading gpt2 model and tokenizer...


Evaluating:   0%|          | 0/100 [00:00<?, ?it/s]



--------------------------------------------------------------------------------
Example 1:
Question: The bananas at the supermarket cost $0.80 each, or a bunch for $3.00.  Jenny buys 10 bunches that average 4 bananas per bunch. How much money, in dollars, did she save by buying the bananas in bunches instead of individually?
Target answer: 2.0

Full CoT Prompt:
----------------------------------------
Question: For every 12 cans you recycle, you receive $0.50, and for every 5 kilograms of newspapers, you receive $1.50. If your family collected 144 cans and 20 kilograms of newspapers, how much money would you receive?
Answer: There are 144/12 = <<144/12=12>>12 sets of 12 cans that the family collected.
So, the family would receive $0.50 x 12 = $<<0.50*12=6>>6 for the cans.
There are 20/5 = <<20/5=4>>4 sets of 5 kilograms of newspapers that the family collected.
So, the family would receive $1.50 x 4 = $<<1.50*4=6>>6 for the newspapers.
Therefore, the family would receive a total of $

Evaluating:   1%|          | 1/100 [00:08<13:21,  8.10s/it]

...

Evaluating:  14%|█▍        | 14/100 [03:02<14:50, 10.35s/it]


Full Model Response:
----------------------------------------
In our example above we're using 2^32 bytes which means 32 bits or 64bits equals 0x00fffffff64 . We'd need 8192 lines instead so it looks like there was about 909988 rows (or 11) words added onto 1 page compared against what IBM does today... What do I get when putting my last line into @#$%&#($)? When adding another string up front where "" stands between two letters ? Are these numbers equal enough while having no way around rounding down without doing extra work along the diagonal side again?? In other word , don´t use something even close though not quite right! You could try starting over straight ahead until things look good :) The best solution here will be making sure everyone who worked hard during production didn`T lose any time trying new ideas ! It should also help us remember people working really long hours  sometimes aren\' t ready yet too late !!
----------------------------------------

Extracted predicted 

Evaluating:  14%|█▍        | 14/100 [03:44<22:58, 16.02s/it]


Full Model Response:
----------------------------------------
Step 1) Find three numbers equal to (x * 3 - 2). Now add these up so we get our number 8497767893745431333995928571727392429253414695875666584227964557823483898887430956092159010319059061526873112044947689044544553562158118621509098533053594087094074632105304289309560097297117458936076691088090898973109078319902178498205207107606135209042085220194129019088705349763268530799615740955764859870701616726927942506347169559556165149666999479017448665268489659757249579698478696608288721954971893482674062657083682041003248389668258631758073862004051888062989645960969564969764620613925901524470638738812735517741128018667998997605138808335299153179147805168687759778378558339000599314187229647227238235270264286336277488679798889487164190704180508338184384118476255334234134137356136294115306133228114285276176256308275688296156126146155246266103284132166154162148142143141125104102113101112110111106010202120011001111999900008018001000900

example_idx,▁▂▂▃▃▄▄▅▅▆▆▇▇█
running_accuracy,█▄▃▂▂▂▂▁▁▁▁▁▁▁
error,cannot convert float...
example_idx,13
running_accuracy,0.07143
